# Classification : 2 Class

## 1.환경준비

### (1) 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.backend import clear_session
from tensorflow.keras.optimizers import Adam

In [ ]:
# 함수로 만들어서 사용합시다.
def dl_history_plot(history):
    plt.figure(figsize=(10,6))
    plt.plot(history['loss'], label='train_err')
    plt.plot(history['val_loss'], label='val_err')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

### (2) 데이터로딩

In [ ]:
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/titanic.3.csv"
data = pd.read_csv(path)
data.drop(['Age_scale1', 'AgeGroup', 'SibSp','Parch' ], axis = 1, inplace = True)
data.head()

## 2.데이터 준비

Sex, Age, Fare 만 이용하여 Survived 를 예측하는 모델을 만들어 봅시다.

### (1) 데이터 분할1 : x, y 나누기

In [ ]:
target = 'Survived'
features = ['Sex', 'Age', 'Fare']
x = data.loc[:, features]
y = data.loc[:, target]

### (2) 변수정리

### (3) NA 조치

### (4) 가변수화

In [ ]:
x = pd.get_dummies(x, columns = ['Sex'], drop_first = True)
x.head()

### (5) 데이터분할2

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.3, random_state = 2022)

### (6) Scaling

In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

## 3.Logistic Regression vs. DL

### (1) Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression()

model1.fit(x_train, y_train)
print(list(x))
model1.coef_, model1.intercept_

In [ ]:
pred1 = model1.predict(x_val)

### (2) DL

* 모델 설계

In [ ]:
nfeatures = x_train.shape[1] #num of columns
nfeatures

In [ ]:
# 메모리 정리
clear_session()

# Sequential 타입 모델 선언
model2 = Sequential()

# Layer 정의
layer = Dense(1, input_shape = (nfeatures,), activation= 'sigmoid')

# 모델에 추가
model2.add(layer)

# 모델요약
model2.summary()

* compile + 학습

In [ ]:
model2.compile(optimizer=Adam(lr=0.01), loss='binary_crossentropy')

history = model2.fit(x_train, y_train, 
                    epochs = 60,
                    validation_split=0.2).history

* 학습결과 그래프

In [ ]:
dl_history_plot(history)

In [ ]:
pred2 = model2.predict(x_val)

# activation이 sigmoid --> 0 ~ 1 사이의 확률값.
# 그러므로 cut-off value(보통 0.5)를 기준으로 잘라서 0과 1로 만들어 준다.
pred2 = np.where(pred2 >= .5, 1, 0)

### (3) 모델 비교

* 가중치 비교

In [ ]:
# 로지스틱 회귀
model1.coef_, model1.intercept_

In [ ]:
# DL
model2.layers[0].get_weights()

* 성능비교

In [ ]:
# 로지스틱 회귀
print(classification_report(y_val, pred1))

In [ ]:
# DL
print(classification_report(y_val, pred2))

## 4.DL1
* 이제 전체 데이터를 가지고 모델링을 시도해 보겠습니다.


### (1) 데이터 전처리

* 데이터 분할1

In [ ]:
target = 'Survived'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

* 가변수화

In [ ]:
cat_cols = ['Pclass','Sex', 'Embarked']
x = pd.get_dummies(x, columns = cat_cols, drop_first = True)

* 데이터분할2

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=.3, random_state = 2022)

* 스케일링

In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

### (2) 모델링

* 모델 설계

In [ ]:
nfeatures = x_train.shape[1] #num of columns
nfeatures

In [ ]:
# 메모리 정리


# Sequential 타입 모델 선언


# Layer 정의


# 모델에 추가


# 모델요약


* compile + 학습

* 학습결과 그래프

* 예측 및 평가

## 5.DL2 : hidden layer
* 이제 레이어를 추가해 보겠습니다.


### (1) 모델 설계

In [ ]:
nfeatures = x_train.shape[1] #num of columns
nfeatures

In [ ]:
# 메모리 정리
clear_session()

# Sequential 타입 모델 선언
model = Sequential()

# Layer 정의
layer1 = Dense(8, input_shape = (nfeatures,), activation = 'relu')
layer2 = Dense(1, activation = 'sigmoid')

# 모델에 추가
model.add(layer1)
model.add(layer2)

# 모델요약
model.summary()

### (2) compile + 학습

In [ ]:
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy')

history = model.fit(x_train, y_train
                     , epochs = 100, batch_size=32
                     , validation_split=0.2).history

* 학습결과 그래프

In [ ]:
dl_history_plot(history)

* 예측 및 평가

In [ ]:
pred = model.predict(x_val)
pred = np.where(pred >= .5, 1, 0)
print(classification_report(y_val, pred2))

### (3) 실습1

* 다음의 summary를 보고 모델을 설계하시오.

| Layer (type) | Output Shape | 옵션 |
| ---- | ---- | ---- | 
| dense (Dense) | (None, 16)  | node, input_shape, activation = 'relu' |
| dense_1 (Dense) | (None, 1) | node, activation = 'sigmoid' |     

In [ ]:
# 메모리 정리


# Sequential 타입 모델 선언


# Layer 정의


# 모델에 추가


# 모델요약


### (4) 실습2

* 다음의 summary를 보고 모델을 설계하시오.

| Layer (type) | Output Shape |  옵션 |
| ---- | ---- | ---- | 
| dense (Dense) | (None, 16) |  node, input_shape, activation = 'relu' |
| dense_1 (Dense) | (None, 8) |  node, activation = 'relu'  |
| dense_2 (Dense) | (None, 1) |  node, activation = 'sigmoid'  |

In [ ]:
# 메모리 정리


# Sequential 타입 모델 선언


# Layer 정의


# 모델에 추가


# 모델요약


### (5) 실습3

* 이번에는 여러분이 원하는 대로 설계하고, 학습해 봅시다.


In [ ]:
# 메모리 정리


# Sequential 타입 모델 선언


# Layer 정의


# 모델에 추가


# 모델요약
